# Visualization Project

---
**Authors**:

- Juan Pablo Zaldivar
- Enric Millán
---

## Introduction

In this second project, the focus is on analyzing collision data in New York City during the summer of 2018. The primary objective is to develop a comprehensive interactive visualization that can address several key questions regarding the nature and patterns of these collisions. With the use of datasets related to collisions, weather conditions, and the New York City map, we aim to explore various facets, including ...

This file contains all the steps required to ensure reproducibility of steps leading from raw data to a clean dataset. The project is divided in three parts: the first part corresponds to the preprocessing of the data, the second part corresponds to the visualization desing process and the third part corresponds to the implementation of the visualization in the streamlit app to answer the questions.

The datasets are as follows:

- Collision Dataset: Extracting and filtering collision data specifically from June to September of 2018. This involves selecting relevant columns, handling missing or inconsistent data, and ensuring data quality.

- Weather Dataset: Locating and incorporating weather data corresponding to the time frames and areas of interest.

- New York City Map: Acquiring a suitable map of New York City to overlay geographical information related to collision locations.

### Dataset obtention and description

The Collisions dataset (`collisions_2018-2020.csv`) was extracted from the former project. The Motor Vehicle Collisions crash table contains details on the crash event. Each row represents a crash event. The Motor Vehicle Collisions data tables contain information from all police reported motor vehicle collisions in NYC. The dataset has to be preprocessed again in order to meet the new specifications.

The weather dataset (`weather2018.csv`) was already given by the supervisors of the project. It contains the weather conditions of the city of New York during the summer of 2018.

The map dataset was obtained from this [cartography web page](https://cartographyvectors.com/map/508-new-york-city-boroughs-ny).

The datasets are located in the folder `Data/`. Following are the loading of each dataset and the import of the required libraries.

### Libraries

For the correct functionality of the executions, the following folders and all their files are needed:

- `Data/`: Folder containing the datasets.
- `Modules/`: Folder containing the modules used in the project.

In [45]:
# pip install altair==5.1.2 pandas==1.5.3 numpy==1.23.5 altair==5.1.2 h3pandas==0.2.5 geopandas==0.13.2 vegafusion[embed]>=1.4.0

In [46]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point 

In [47]:
dir = './Data'

## Dataset Preprocessing

The preprocessing of the files involved a collaborative effort using both OpenRefine and selected Python libraries. This strategic approach was adopted to take advantage of the unique strengths and capabilities offered by each tool. OpenRefine facilitated initial data cleaning and transformation tasks, with its user-friendly interface for effective manipulation of datasets. Simultaneously, Python libraries were utilized to perform more complex data operations and manipulations, with special emphasis on the extensive functionalities and flexibility they provide. This combination allowed for a comprehensive preprocessing workflow that maximized efficiency and accuracy in preparing the data for subsequent analyses and visualization tasks.

### Collision Dataset Preprocessing

The initial step involved loading the original dataset into a `Pandas` dataframe, primarily to apply a date range filter efficiently. The rationale behind this approach was to optimize the data filtering process, considering the considerable size of the original dataset. The volume of data posed challenges within OpenRefine, leading to slow and inefficient computational processes. By filtering the dataset using `Pandas`, it allowed for a more streamlined and quicker selection of the desired date range. Following this initial filtering phase, the refined dataset was exported as a `.csv` file (`collisions-2018.csv`) and subsequently imported into OpenRefine for further data processing and cleaning procedures. This sequential approach ensured a balance between computational efficiency and data handling capabilities across both Pandas and OpenRefine, resulting in a more effective preprocessing workflow.

In [48]:
collisions = pd.read_csv(dir + '/collisions_2018-2020.csv')
collisions.head(1)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2020-09-06,18:05,NaN,NaN,40.771038,-73.83413,"(40.771038, -73.83413)",WHITESTONE EXPRESSWAY,NaN,NaN,...,Following Too Closely,NaN,NaN,NaN,4345591,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN


In [49]:
collisions['YEAR'] = pd.DatetimeIndex(collisions['CRASH DATE']).year
collisions.shape

(115740, 30)

In [50]:
collisions = collisions[collisions['YEAR'] == 2018]
collisions.shape

(79383, 30)

#### Data Type Conversion

In OpenRefine, the data conversion process involved several attribute adjustments. The CRASH DATE attribute underwent a conversion to a date type for enhanced consistency and data clarity. Meanwhile, both COLLISION ID and CRASH TIME were temporarily set as string types.

Attributes pertaining to the geographical location of the collisions were modified to strings, accompanied by specific notations. As part of this process, all values were standardized to uppercase, and any extra spaces were removed where applicable. This standardization was implemented to facilitate the effectiveness of the clustering method utilized for collectively inspecting and modifying cells. The objective was to streamline the identification and correction of any inconsistencies or inaccuracies within the data, ensuring a more uniform and reliable dataset for subsequent analyses.

The attributes pertaining to the number of persons involved in the collision underwent a data type conversion to integers within the dataset. This decision was driven by the discrete nature of these values and the expectation that these numerical counts wouldn't contain negative values.

Conversely, the attributes related to vehicles and factors involved in the collisions were retained as strings temporarily. This choice was made to maintain flexibility in handling these attributes during subsequent data processing and analysis phases, ensuring that any necessary modifications or categorizations could be applied effectively as the analysis progressed.

#### Data Selection and Transformation

From previous knowledge of the dataset, a subset of attributes was selected for further analysis. This selection was based on the relevance of the attributes to the research questions and the availability of data.

In [51]:
interest_cols = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'ON STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'COLLISION_ID', 'VEHICLE TYPE CODE 1']

collisions = collisions[interest_cols]

**Geographic attributes**

As we saw in the previous project, **ON STREET NAME** and **OFF STREET NAME** seem to be the same attribute, but with different names. The web site of the dataset cointains the following descriptions:

- **ON STREET NAME**: *Street on which the collision occurred*.
- **OFF STREET NAME**: *Street address if known*.

This gives the idea that both attributes probably contain the same information. Furthermore, there are no rows with both attributes filled, which makes the idea of merging both attributes plausible and would consolidate information without redundancy.

In [52]:
collisions[(collisions['ON STREET NAME'].notnull()) & (collisions['OFF STREET NAME'].notnull())].shape

(0, 19)

In [53]:
collisions[(collisions['ON STREET NAME'].notnull()) | (collisions['OFF STREET NAME'].notnull())].shape

(79157, 19)

The resulting attribute after merging both columns is called **STREET NAME** and contains the street name/address where the collision occurred, with no missing values. Some rows will have a more detailed description of the street, while others will only have the name of the street. The fusion of the two columns will be done in OpenRefine.

The clustering process utilized the key collision method in conjunction with the fingerprint keying function, applied separately to each individual geographical attribute. After a couple of iterations, no substantial alterations in attribute values were identified. However, during this process, misspellings were detected and rectified to ensure data accuracy.

The misspellings were corrected and the clusterization was done again. The results were the same, which means that the values were already consistent. To verify the result, a Neares Neighbours analysis was done as well but without finding any significant variation.

**Vehicle attributes**

We have seen already that there are many classes of the VEHICLE CODE TYPE 1 values. To simplify this complexity, the project statement force to reduce the diversity of classes by employing a clustering technique. This clustering methodology allowed us to condense the multitude of classes within VEHICLE CODE TYPE 1 into a more manageable set of clusters, facilitating a more comprehensible and concise representation for subsequent visualization and analysis purposes.

In [54]:
# PASAR ESTA FUNC A PREPROCESSING

collisions['VEHICLE TYPE CODE 1'] = collisions['VEHICLE TYPE CODE 1'].replace(['Taxi', 'TAXI'], 'TAXI')
collisions['VEHICLE TYPE CODE 1'] = collisions['VEHICLE TYPE CODE 1'].replace(['Fire', 'FD tr', 'firet', 'fire', 'FIRE', 'fd tr', 'FD TR', 'FIRE', 'FIRET'], 'FIRE')
collisions['VEHICLE TYPE CODE 1'] = collisions['VEHICLE TYPE CODE 1'].replace(['AMBUL', 'Ambulance', 'ambul', 'AMB', 'Ambul', 'AMBULANCE', 'AMBU'], 'AMBULANCE')

The resulting classes are the following:

- **TAXI**: 
- **FIRE**:
- **AMBULANCE**:

A similar strategy was done with the CONTRIBUTING FACTOR VEHICLE 1 attribute. However, the aggregation was not so exhaustive since this attribute wasn't needed a priori for the main questions that the visualizations should answer. For this attribute basic merge transformations were applied in OpenRefine until no "strange" or "uninformative" nor repeated classes remained.

**Number of persons attributes**

For the visualization purposes, the differentantion of **PERSONS**, **PEDESTRIANS**, **CYCLISTS** and **MOTORISTS** (**INJURED/KILLED**) is irrelevant. A more useful attribute would be the total number of persons involved in the collision. This can be obtained by summing the four attributes under the assumption that the **PERSONS** attribute is not the sum of the other three attributes.

This condition was needed to be checked because the documentation of the dataset was not precise enough to determinate if **NUMBER OF PERSON INJURED/KILLED** was an aggregate from the other three columns or not.

*Note: The metadata information available in the web of the dataset was: "Number of persons injured/killed" regarding the **NUMBER OF PERSONS INJURED/KILLED**.*

In [55]:
collisions['NUMBER OF PERSONS INJURED'].equals(collisions['NUMBER OF PEDESTRIANS INJURED'] + collisions['NUMBER OF CYCLIST INJURED'] + collisions['NUMBER OF MOTORIST INJURED'])

False

In [56]:
collisions['NUMBER OF PERSONS INJURED'].equals(collisions['NUMBER OF PEDESTRIANS INJURED'])

False

In [57]:
collisions['NUMBER OF PERSONS KILLED'].equals(collisions['NUMBER OF PEDESTRIANS KILLED'] + collisions['NUMBER OF CYCLIST KILLED'] + collisions['NUMBER OF MOTORIST KILLED'])

False

In [58]:
collisions['NUMBER OF PERSONS KILLED'].equals(collisions['NUMBER OF PEDESTRIANS KILLED'])

False

As seen by the logical comprobations, the **NUMBER OF PERSONS INJURED/KILLED** is not the sum of the other three attributes. Furthermore, the terms persons and pedestrians are not equal, as one could have thought that the term persons was used to refer to pedestrians.

Based on this, the discrete attributes refering to the injured people were summed to obtain **NUMBER OF INJURED** and the discrete attributes refering to the killed people were summed to obtain **NUMBER OF KILLED**. The **NUMBER OF INJURED/KILLED** attributes were removed.

In [59]:
# PASAR ESTA FUNC A PREPROCESAMIENTO

collisions['TOTAL INJURED'] = collisions.filter(regex='INJURED').sum(axis=1)
collisions['TOTAL KILLED'] = collisions.filter(regex='KILLED').sum(axis=1)

collisions = collisions.drop(collisions.iloc[:, 8:16], axis=1)

---

In [60]:
collisions.to_csv(dir + '/collisions-2018.csv', index=False)

In [61]:
collisions = pd.read_csv(dir + '/collisions-2018_prepro_v1.csv')

In [62]:
collisions = collisions[collisions['VEHICLE TYPE CODE 1'].isin(['FIRE', 'TAXI', 'AMBULANCE'])]

collisions['VEHICLE TYPE CODE 1'].unique()

array(['TAXI', 'AMBULANCE', 'FIRE'], dtype=object)

In [63]:
collisions.shape

(4092, 12)

At this point, the dataset contains the attributes needed (with the weather attributes as an exception) for the analysis and some extra attributes that were considered interesting for some possible extra analysis or insights that we could think about.

---

#### Missing Attributes

It has already been mentioned the existence of some missing values. In the previous section, the verification of missing values was done with the .isnull() method of Pandas. However, this method does not take into account the `NaN` values. In order to check the existence of NaN values, the .isna() method was used.

In [64]:
comp = (collisions.isnull().sum() == collisions.isna().sum())
comp[comp == False]

Series([], dtype: bool)

As seen previously, all the missing values of the dataset are detected both with .isnull() and .isna(). After this check, notice that the only attributes with missign values are corresponding to geographical properties of the collisions. We can apply a similar strategy as the one used in the previous project to fill the missing values.

In [65]:
collisions.isnull().sum()

COLLISION_ID                        0
CRASH DATE                          0
CRASH TIME                          0
BOROUGH                          1385
ZIP CODE                         1385
LATITUDE                          292
LONGITUDE                         292
STREET NAME                         0
CONTRIBUTING FACTOR VEHICLE 1       0
VEHICLE TYPE CODE 1                 0
TOTAL INJURED                       0
TOTAL KILLED                        0
dtype: int64

In [66]:
292*100/collisions.shape[0]

7.135874877810362

The number of missing values is least for the coordinates values, with $7.14\%$ of the rows missing. The idea to impute the missing values is by checking if a point specified by its coordinates (LONGITUDE, LATITUDE) falls within the boundary polygon of different BOROUGH values. If the original attribute value is null, this method aims to assign the specific borough corresponding to the geographic location of the provided coordinates.

In [67]:
nyc_map = gpd.read_file('Data/new-york-city-boroughs-ny_.geojson')

boroughs = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

# Extraction of the borough polygon
borough_poly = {}
for b in boroughs:
    poly = nyc_map[nyc_map['name'] == b]['geometry']
    borough_poly[b] = poly.values[0]

In [68]:
for idx, row in collisions.iterrows():
    if row['BOROUGH'] is not None:
        lon = row['LONGITUDE']
        lat = row['LATITUDE']

        if lat is not None and lon is not None:
            p = Point(lon, lat)
            for b, poly in borough_poly.items():
                if p.within(poly):
                    collisions.loc[idx, 'BOROUGH'] = b.upper()
                    break

c:\Users\jpniz\OneDrive\Documentos\UPC\Q5\.conda\Lib\site-packages\shapely\predicates.py:946: RuntimeWarning: invalid value encountered in within
  return lib.within(a, b, **kwargs)


In [69]:
collisions.isnull().sum()

COLLISION_ID                        0
CRASH DATE                          0
CRASH TIME                          0
BOROUGH                           174
ZIP CODE                         1385
LATITUDE                          292
LONGITUDE                         292
STREET NAME                         0
CONTRIBUTING FACTOR VEHICLE 1       0
VEHICLE TYPE CODE 1                 0
TOTAL INJURED                       0
TOTAL KILLED                        0
dtype: int64

We observe a considerably lower number of missing values in the BOROUGH attribute compared to the missing values in the coordinates.